# Identification of Quality Models from data

In [ ]:
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt

import sys

sys.path.append('C:/Users/LocalAdmin/Documents/GitHub/DigitalTwinInjectionMolding/')
sys.path.append('E:/GitHub/DigitalTwinInjectionMolding/')
sys.path.append('/home/alexander/GitHub/DigitalTwinInjectionMolding/')

from DIM.miscellaneous.PreProcessing import arrange_data_for_ident

from DIM.models.model_structures import GRU
from DIM.models.injection_molding import QualityModel
from DIM.optim.param_optim import ModelTraining

Load experimental data, use 5 cycles for parameter estimation (model training) and 3 cycles for model validation 

In [ ]:
cycles = []

for i in range(1,11):
    cycles.append(pkl.load(open('../data/Versuchsplan/cycle'+str(i)+'.pkl','rb')))

cycles_train = cycles[2:7]
cycles_val = cycles[7:10]

In [ ]:
print(cycles[0])

Select process variables and product quality measurements for modeling

In [ ]:
q_lab = ['Durchmesser_innen']
x_lab= [['p_wkz_ist','T_wkz_ist','p_inj_ist','Q_Vol_ist','V_Screw_ist']]

In [ ]:
data_train,init_train,switch_train = arrange_data_for_ident(cycles_train,q_lab,x_lab,'quality')
data_val,init_val,switch_val = arrange_data_for_ident(cycles_val,q_lab,x_lab,'quality')

Plot training data

In [ ]:
fig, axs = plt.subplots(2,3)
fig.set_size_inches((40/2.54,20/2.54))
plt.subplot(2,3,1,title='cycle2',xlabel='k');plt.plot(data_train[0][x_lab[0]])
plt.subplot(2,3,2,title='cycle3',xlabel='k');plt.plot(data_train[1][x_lab[0]])
plt.subplot(2,3,3,title='cycle4',xlabel='k');plt.plot(data_train[2][x_lab[0]])
plt.subplot(2,3,4,title='cycle5',xlabel='k');plt.plot(data_train[3][x_lab[0]])
plt.subplot(2,3,5,title='cycle6',xlabel='k');plt.plot(data_train[4][x_lab[0]])

plt.subplot(2,3,6,title='D innen',xlabel='cycle')
plt.plot([2,3,4,5,6],[data_train[i][q_lab].iloc[0].values for i in range(len(data_train))],'d',markersize=12,label='train true')
plt.plot([7,8,9],[data_val[i][q_lab].iloc[0].values for i in range(len(data_val))],'d',markersize=12,label='val true')
plt.legend()
plt.subplots_adjust(hspace=0.3)
plt.show()

Initialize Quality Model, comprising one model for the injection, pressure and cooling phase, respectively

In [ ]:
dim_c = 2

injection_model = GRU(dim_u=5,dim_c=dim_c,dim_hidden=10,dim_out=1,name='inject')
press_model = GRU(dim_u=5,dim_c=dim_c,dim_hidden=10,dim_out=1,name='press')
cool_model = GRU(dim_u=5,dim_c=dim_c,dim_hidden=10,dim_out=1,name='cool')

quality_model = QualityModel(subsystems=[injection_model,press_model,cool_model],
                              name='q_model')

In [ ]:
c0_train = [np.zeros((dim_c,1)) for i in range(0,len(x_train))]
c0_val = [np.zeros((dim_c,1)) for i in range(0,len(x_val))]

data = {'u_train': x_train,
        'y_train': q_train,
        'switch_train': switch_train,
        'init_state_train': c0_train,
        'u_val': x_val,
        'y_val': q_val,
        'switch_val': switch_val,
        'init_state_val': c0_val}

Usually the model and the data is given to the ModelTraining() procedure, to estimate the optimal parameters, this has been done in advance so the results are merely loaded by calling pkl.load()

In [ ]:
results = ModelTraining(quality_model,data)
results = pkl.load(open('../QualityModel_GRU_1c_5in_1out.pkl','rb'))
print(results)

The parameters the led to the best results on the validation dataset (row index 3) are assigned to the quality model

In [ ]:
quality_model.AssignParameters(results.loc[3,'params'])

Evaluate the trained model on the training data and the validation data, to see how well the model predicts product quality from process measurements

In [ ]:
y_train = []
y_val = []

#Estimation on training data cycles
for i in range(0,5): 
    _,y = quality_model.Simulation(data['init_state_train'][i], data['u_train'][i],None,data['switch_train'][i])
    y = np.array(y[-1])[0,0]
    y_train.append(y)

#Estimation on validation data cycles    
for i in range(0,3): 
    _,y = quality_model.Simulation(data['init_state_val'][i], data['u_val'][i],None,data['switch_val'][i])
    y = np.array(y[-1])[0,0]
    y_val.append(y)

Plot input validation data as well as predicted and true output data

In [ ]:
fig, axs = plt.subplots(2,2)
fig.set_size_inches((40/2.54,20/2.54))
plt.subplot(2,2,1,title='cycle 7',xlabel='k');plt.plot(x_val[0])
plt.subplot(2,2,3,title='cycle 8',xlabel='k');plt.plot(x_val[1])
plt.subplot(1,2,2,title='D innen',xlabel='cycle')

plt.plot([2,3,4,5,6],q_train,'d',markersize=12,label='train true')
plt.plot([7,8,9],q_val,'d',markersize=12,label='val true')
plt.plot([2,3,4,5,6],y_train,'d',markersize=12,label='train est')
plt.plot([7,8,9],y_val,'d',markersize=12,label='val est')
plt.subplots_adjust(hspace=0.3 )
plt.legend()
plt.show()

I.e. the model overestimates the true quality measurement "D innen" 